In [1]:
import os
import pandas as pd
import plotly.express as px
import requests
import sys
sys.path.append('C:/Users/Sreynoch/pythonlab/Lib/site-packages')

from celery import Celery

from celery.schedules import crontab

ModuleNotFoundError: No module named 'celery'

In [ ]:
# Define the path to the Excel file
EXCEL_FILE_PATH = r'D:\AI_TUX\Python\AutoMateOfficeTask2\interactive_chart_project\Financial_Data.xlsx'  # Modify as per your path

# Initialize Celery
app = Celery('tasks', broker='redis://localhost:6379/0')

# Function to process data from Excel and generate a chart
def process_data_and_generate_chart(file_path):
    """
    Reads Excel data, processes it, and generates an interactive chart.
    Saves the chart as an HTML file in the same directory.
    """
    # Load the data from the Excel file
    data = pd.read_excel(file_path)

    # Check if required columns exist
    if 'Category' not in data.columns or 'Sales' not in data.columns:
        raise ValueError("The Excel file must contain 'Category' and 'Sales' columns.")
    
    # Summarize the sales data by category
    summary = data.groupby('Category')['Sales'].sum().reset_index()

    # Create a bar chart using Plotly
    fig = px.bar(summary, x='Category', y='Sales', title='Sales by Category',
                 labels={'Category': 'Category', 'Sales': 'Total Sales'},
                 color_discrete_sequence=["#1f77b4"])

    # Save the chart as an HTML file in the same directory
    output_path = os.path.join(os.path.dirname(file_path), 'sales_by_category.html')
    fig.write_html(output_path)

    return output_path  # Return the path to the saved chart

# Function to send the chart to Gemini API
def send_chart_to_gemini_api(chart_path):
    """
    Sends the generated chart to the Gemini API for further use.
    """
    # Example Gemini API URL - replace with actual endpoint
    url = "https://api.gemini.com/v1/chart/upload"
    
    # Read the file and send it in a POST request
    with open(chart_path, 'rb') as f:
        response = requests.post(url, files={'file': f})

    # Return the response from the API
    return response.json()

# Define a Celery task to process the data and send the chart to Gemini
@app.task
def generate_weekly_chart():
    """
    Celery task that generates a weekly chart and sends it to the Gemini API.
    """
    chart_path = process_data_and_generate_chart(EXCEL_FILE_PATH)
    response = send_chart_to_gemini_api(chart_path)
    return response

# Schedule the Celery task to run every Monday at 8:00 AM
@app.on_after_configure.connect
def setup_periodic_tasks(sender, **kwargs):
    """
    Configures Celery to run the 'generate_weekly_chart' task every Monday at 8:00 AM.
    """
    sender.add_periodic_task(
        crontab(hour=8, minute=0, day_of_week='mon'),
        generate_weekly_chart.s()
    )